In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
# Shell Plus Model Imports
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission, User
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
# Shell Plus Django Imports
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, Exists, OuterRef, Subquery
from django.utils import timezone
from django.urls import reverse

Drug Inventory Models
- cmsinv.InventoryItem -> inventory_item
    - id, stock_qty

In [3]:
from cmsinv.models import (
    InventoryItem, Prescription, PrescriptionDetail, InventoryMovementLog,
    DepletionItem, ReceivedItem
)

In [4]:
# Get all Prescribed drug ids
prescribed_set = set()
for record in PrescriptionDetail.objects.all():
    prescribed_set.add(record.drug.id)
print(f"Prescribed count: {len(prescribed_set)}")

# Get all Received drug ids
received_set = set()
for received in ReceivedItem.objects.all():
    received_set.add(received.drug_item_id)
print(f"Received count: {len(received_set)}")

# Get all Depleted drug ids
depleted_set = set()
for depleted_item in DepletionItem.objects.all():
    depleted_set.add(depleted_item.drug_id)
print(f"Depleted count: {len(depleted_set)}")

Prescribed count: 538
Received count: 69
Depleted count: 460


In [5]:
mark_for_delete = InventoryItem.objects.filter(
    stock_qty=0,
    last_updated__isnull=True,
    clinic_drug_no__isnull=True,
).filter(
    Q(updated_by__isnull=True) |
    Q(updated_by__exact='')
)
marked_set = set()
for item in mark_for_delete:
    marked_set.add(item.id)
print(f"Marked set count: {len(marked_set)}")

Marked set count: 6112


In [6]:
utilized_set = prescribed_set.union(received_set).union(depleted_set)
print(len(utilized_set))

597


In [7]:
# Exclude utlized set from marked set
final_set = marked_set.difference(utilized_set)
print(f"Final set count: {len(final_set)}")

Final set count: 6108


In [8]:
intersect = marked_set.intersection(utilized_set)
for item in intersect:
    if item in final_set:
        print(f'Mark item #{item} for delete')
    else:
        print(f'Item #{item} excluded from final_set')

Item #19819 excluded from final_set
Item #21827 excluded from final_set
Item #5398 excluded from final_set
Item #5151 excluded from final_set


In [9]:
for item_id in final_set:
    item = InventoryItem.objects.get(pk=item_id)
    print(f"#{item.id:<6}|{item.discontinue}|{item.product_name}")
    item.location = "Trash"
    item.remarks = "Mark for deletion"
    item.delete()

#19964 |True|0.3% W/V POTASSSIUM CHLORIDE AND 0.9% SODIUM CHLORIDE IV INFUSION
#19965 |True|0.9% SODIUM CHLORIDE INJECTION
#19966 |True|0.9% SODIUM CHLORIDE INJECTION, USP IN MINI-BAG PLUS CONTAINER
#19967 |True|A1 CREAM 1%
#19968 |True|A1-N COUGH SYRUP
#19969 |True|ABACOD COD LIVER OIL CAP
#19970 |True|ABBLE SYRUP
#19971 |True|ABILIFY DISCMELT ORALLY DISINTEGRATING TAB 10MG
#19972 |True|ABILIFY DISCMELT ORALLY DISINTEGRATING TAB 15MG
#19973 |True|ABILIFY INJECTION 9.75MG/1.3ML
#19974 |True|ABILIFY TABLETS 2MG
#19975 |True|A-BITE CREAM 10%
#19976 |True|ABRAXANE POWDER FOR INJECTABLE SUSPENSION 100MG
#19977 |True|ACARBOSE FARMOZ TAB 100MG
#19978 |True|ACARBOSE FARMOZ TAB 100MG
#19979 |True|ACARBOSE FARMOZ TAB 50MG
#19980 |True|ACARBOSE FARMOZ TAB 50MG
#19981 |True|ACARBOSE TAB 50MG
#19982 |True|AC-B COMPLEX PLUS C TABLETS
#19983 |True|ACCORD IMATINIB TABLET 100MG
#19984 |True|ACCORD IMATINIB TABLET 400MG
#19985 |True|ACCORD ZOLEDRONIC ACID CONCENTRATE FOR SOLUTION FOR INFUSION 4MG/5ML
#

#20181 |True|AMERIDE TAB 2MG
#20182 |True|AMGICOL TABLETS  30MG (BLISTER)
#20183 |True|AMGICOL TABLETS 30MG (BOTTLE)
#20184 |True|AMICAN TABLETS 16MG
#20185 |True|AMICAN TABLETS 8MG
#20186 |True|AMIKACIN NORMON SOLUTION FOR INJECTION 250MG/2ML
#20187 |True|AMIKACIN NORMON SOLUTION FOR INJECTION 500MG/2ML
#20188 |True|AMINOGEN-S SOLUTION FOR INFUSION
#20189 |True|AMINOLEBAN 8% AMINO ACID INJ
#20190 |True|AMINOPHYLLINE INJ 250MG/10ML
#20191 |True|AMINOPLASMAL B. BRAUN 10% E SOLUTION FOR INFUSION
#20192 |True|AMINOVEN 10% SOLUTION FOR INFUSION
#20193 |True|AMIODARONE SANDOZ TAB 200MG
#20194 |True|AMISULPRIDE NORMON FILM-COATED TAB 100MG
#20195 |True|AMISULPRIDE NORMON FILM-COATED TAB 200MG
#20196 |True|AMISULPRIDE NORMON FILM-COATED TAB 400MG
#20197 |True|AMISULPRIDE SANDOZ TABLETS 100MG
#20198 |True|AMISULPRIDE SANDOZ TABLETS 200MG
#20199 |True|AMISULPRIDE SANDOZ TABLETS 400MG
#20200 |True|AMK TAB 1000MG
#20201 |True|AMK TAB 375MG
#20202 |True|AMK TAB 625MG
#20203 |True|AMLOBIN TAB 5MG "

#20387 |True|APO-HYDROCHLOROTHIAZIDE TAB 50MG
#20388 |True|APO-IRBESARTAN TABLETS 150MG
#20389 |True|APO-IRBESARTAN TABLETS 300MG
#20390 |True|APO-LAMIVUDINE HBV TABLETS 100MG
#20391 |True|APO-LAMIVUDINE TABLETS 150MG
#20392 |True|APO-LAMIVUDINE  TABLETS 300MG
#20393 |True|APO-LAMIVUDINE-ZIDOVUDINE TABLETS 150MG/300MG
#20394 |True|APO-LAMOTRIGINE TAB 25MG
#20395 |True|APO-LANSOPRAZOLE DELAYED-RELEASE CAPSULES 15MG
#20396 |True|APO-LANSOPRAZOLE DELAYED-RELEASE CAPSULES 30MG
#20397 |True|APO-LETROZOLE TABLETS 2.5MG
#20398 |True|APO-LEVOFLOXACIN TAB 250MG
#20399 |True|APO-LISINOPRIL/HCTZ TAB 20/12.5MG
#20400 |True|APO-LITHIUM CARBONATE CAPSULES 150MG
#20401 |True|APO-LITHIUM CARBONATE CAPSULES 300MG
#20402 |True|APO-LORAZEPAM TABLETS 1MG
#20403 |True|APO-LORAZEPAM TABLETS 2MG
#20404 |True|APO-LOSARTAN TABLETS 100MG
#20405 |True|APO-LOSARTAN TABLETS 50MG
#20406 |True|APO-LOSARTAN/HCTZ TABLETS 100MG/25MG
#20407 |True|APO-LOSARTAN/HCTZ TABLETS 50MG/12.5MG
#20408 |True|APO-MEDROXY TAB 5MG
#20

#20586 |True|ATV-10 TAB 10MG
#20587 |True|ATV-20 TAB 20MG
#20588 |True|AUBAGIO TAB 14MG
#20589 |True|AUGITAR LIQUID 1%
#20590 |True|AUGMENTIN TAB 375MG
#20591 |True|AURASERT 50 TAB 50MG
#20592 |True|AUROMET 850 - SR TAB 850MG
#20593 |True|AUROPRAM 10 TAB 10MG
#20594 |True|AUROPRAM 20 TAB 20MG
#20595 |True|AUROPRAM 40 TAB 40MG
#20596 |True|AUROTAZ-P 2.25 FOR INJ 2.25GM
#20597 |True|AUROTAZ-P 4.5 FOR INJ 4.5GM
#20598 |True|AVANCE LIQUI CAL/MAG CAP
#20599 |True|AVARIN CAPSULES
#20600 |True|AVASTIN ROCHE CONCENTRATE FOR SOLN FOR INFUSION 100MG/4ML
#20601 |True|AVASTIN ROCHE CONCENTRATE FOR SOLN FOR INFUSION 400MG/16ML
#20602 |True|AVAXIM 160U VACCINE
#20603 |True|AVAXIM PEDIATRIC VACCINE 80U
#20604 |True|AVELOX TAB 400MG
#20605 |True|AVENPHYL 30 TAB 30MG
#20606 |True|AVERTI SOLUTION 25MG/5ML
#20607 |True|AVOCA CAUSTIC PENCILS 40%W/W
#20608 |True|AVODART CAPSULES 0.5MG
#20609 |True|AVONEX SOLUTION FOR INJECTION 30MCG/0.5ML
#20610 |True|AVONEX SOLUTION FOR INJECTION IN PRE-FILLED PEN 30MCG/0

#20804 |True|BIOSONIDE NASAL SPRAY 100MCG/DOSE
#20805 |True|BIOTOR-10 TABLETS 10MG
#20806 |True|BIOTOR-20 TABLETS 20MG
#20807 |True|BISBURG ENTERIC COATED TABLETS 5MG
#20808 |True|BISCO GASTRO-RESISTANT TABLETS 5MG
#20809 |True|BISEDYL TAB ENTERIC-COATED 5MG
#20810 |True|BISOPROLOL STADA FILM-COATED TABLETS 5MG
#20811 |True|BITOMET INJ 1000MCG/ML
#20812 |True|BLACKMORES BIO MAGNESIUM TAB
#20813 |True|BLACKMORES JOINT FORMULA WITH GLUCOSAMINE & CHONDROITIN CAP
#20814 |True|BLACKMORES VITAMIN D3 CAP 1000IU
#20815 |True|BLINGO CALCIUM 600MG WITH VITAMIN D3 TABLETS
#20816 |True|BLINGO GLUCOSAMINE & CHONDROITIN TABLETS
#20817 |True|BLINGO VITAMIN B1-B6-B12 TABLETS
#20818 |True|BLISSA CALCIUM 600MG WITH VITAMIN D3 CAPSULES
#20819 |True|BLISSA CALCIUM 600MG WITH VITAMIN D3 TABLETS
#20820 |True|BLISSA GLUCOSAMINE PLUS CHONDROITIN TABLETS
#20821 |True|BLISSA VITAMIN B COMPLEX PLUS C TABLETS
#20822 |True|BLISSA VITAMIN B1-B6-B12 TABLETS
#20823 |True|BLISSA VITAMIN C TIME RELEASE TABLETS 1000MG
#

...

#25402 |True|TELMOTENS TABLETS 40MG
#25403 |True|TELMOTENS TABLETS 80MG
#25404 |True|TELOL 100 TAB 100MG
#25405 |True|TELOL 50 TAB 50MG
#25406 |True|TEMOZOLOMIDE CAPSULES 100MG
#25407 |True|TEMOZOLOMIDE CAPSULES 20MG
#25408 |True|TENAFINE CREAM 10MG/G
#25409 |True|TENOCARD-100 TABLETS 100MG
#25410 |True|TENOCARD-50 TABLETS 50MG
#25412 |True|TENOREN 50 TAB 50MG
#25413 |True|TENPRESS 50 TABLET 50MG
#25414 |True|TEPADINA POWDER FOR CONCENTRATE FOR SOLUTION FOR INFUSION 100MG
#25415 |True|TEPADINA POWDER FOR CONCENTRATE FOR SOLUTION FOR INFUSION 15MG
#25416 |True|TERADRIN TAB 2MG
#25417 |True|TERAMIN CREAM 2%
#25418 |True|TERAPRESS-1 TAB 1MG
#25419 |True|TERAPRESS-2 TAB 2MG
#25420 |True|TERASIN TAB 2 MG
#25421 |True|TERBIFIN-250 TAB 250MG
#25422 |True|TERBIN SPRAY 1%
#25423 |True|TERBINAFIN STADA CREAM 1% W/W
#25424 |True|TERBINAFINE ACTAVIS TABLET 250MG
#25425 |True|TERBI-SPRAY 1%
#25426 |True|TERBITACK 250 TAB 250MG
#25427 |True|TERBUTA SYRUP 1.5MG/5ML
#25428 |True|TERFUNG TAB 250MG
#254

#25602 |True|TRIMETAZIDINE STADA MR TAB 35MG
#25603 |True|TRIMETAZIDINE-TEVA PROLONGED-RELEASE TABLETS 35MG
#25604 |True|TRIMEZOLE SUSPENSION
#25605 |True|TRI-N CREAM
#25606 |True|TRIPLE ANTIBIOTIC OINTMENT (FIP)
#25607 |True|TRIPLE ANTIBIOTIC OINTMENT (OSCO)
#25608 |True|TRIPLE ANTIBIOTIC OINTMENT LONGS
#25609 |True|TRIPLE ANTIBIOTIC OINTMENT XTRA
#25610 |True|TRIPLE B FORT VITAMIN B1-B6-B12 TABLETS
#25611 |True|TRIPLE B VITAMIN B1-B6-B12 TAB
#25612 |True|TRIPSYL 10 TABLETS 10MG
#25613 |True|TRIPSYL 25 TABLETS 25MG
#25614 |True|TRI-REGOL COATED TAB
#25615 |True|TRISEQUENS TAB
#25616 |True|TRISONE CREAM
#25617 |True|TRITCOPRESS TABLET 50MG
#25618 |True|TRITIN TABLET 100MG
#25619 |True|TRITTICO TAB 50MG
#25620 |True|TRIVEDON MR EXTENDED RELEASE TABLETS 35MG
#25621 |True|TRIX-DM EYE DROPS
#25622 |True|TROBALT FILM-COATED TABLETS 100MG
#25623 |True|TROBALT FILM-COATED TABLETS 200MG
#25624 |True|TROBALT FILM-COATED TABLETS 300MG
#25625 |True|TROBALT FILM-COATED TABLETS 400MG
#25626 |True|T

#25815 |True|VEPACEL PREPANDEMIC INFLUENZA VACCINE SUSPENSION FOR INJECTION MULTIDOSE VIALS
#25816 |True|VERAGEL-DMS TAB
#25817 |True|VERAPIME POWDER AND SOLVENT FOR SOLUTION FOR INJECTION 1G
#25818 |True|VERTIGON TAB 25MG
#25819 |True|VESONIC (NEW FORMULA) CAPSULES
#25820 |True|VESONICDEX TABLETS 0.5MG
#25821 |True|VESPROCIN-250 TABLETS 250MG
#25822 |True|VETAPRIMA B COMPLEX WITH C TABLETS
#25823 |True|VETAPRIMA CAL MAG PLUS VITAMIN D3 TABLETS
#25824 |True|VETAPRIMA CALCIUM PLUS VITAMIN D3 TABLETS
#25825 |True|VETAPRIMA GLUCOSAMINE PLUS CHONDROITIN TABLETS
#25826 |True|VETAPRIMA VITAMIN C 500MG PLUS ZINC TABLETS
#25827 |True|VETAPRIMA VITAMIN E SOFTGELS CAPSULES 400IU
#25828 |True|VIAGRA ORODISPERSIBLE TABLETS 50MG
#25829 |True|VIALEBEX SOLN FOR INF 200MG/ML (INNOLEDGE, PURPLE LABEL)
#25830 |True|VIALEBEX SOLN FOR INF 200MG/ML (INNOLEDGE, RED LABEL)
#25831 |True|VIALEBEX SOLN FOR INF 200MG/ML (WAI FAT PHARMACEUTICAL)
#25832 |True|VIALEBEX SOLN FOR INF 200MG/ML (WILSON)
#25833 |True|VI

#26009 |True|WAH TAT PATCH 30MG
#26010 |True|WAHTATSY COUGH SYRUP
#26011 |True|WAIFULSIN VITAMIN B1-B6-B12 TABLETS
#26012 |True|WAIYSIN VITAMIN B1-B6-B12 TABLETS
#26013 |True|WAN ZAN ENEMA 15%W/W 'J.C.S.'
#26014 |True|WANSHUHONG FOR IV INJ 40MG
#26015 |True|WARFARIN TABLETS 1MG
#26016 |True|WARFARIN TABLETS 3MG
#26017 |True|WARFARIN TABLETS 5MG
#26018 |True|WARTEC SOLUTION 0.5%W/V
#26019 |True|WATER FOR INJ (JEAN-MARIE)
#26020 |True|WATER FOR INJECTIONS
#26021 |True|WATIONAL (NEW FORMULA) CAPSULES
#26022 |True|WATIONALDEX TABLETS 0.5MG
#26023 |True|WATSONS ANTACID/ANTI-GAS CHEWABLE TAB
#26024 |True|WATSONS VITAMIN C PLUS ZINC EFFERVESCENT TAB
#26025 |True|WEBBER NATURALS GLUCOSAMINE CHONDROITIN SULFATE 250/200MG CAP
#26026 |True|WEBBER NATURALS GLUCOSAMINE CHONDROITIN SULFATE EXTRA STRENGTH 500/400MG CAP
#26027 |True|WEBBER NATURALS GLUCOSAMINE SULFATE SODIUM FREE CAP 500MG
#26028 |True|WEIDAREN GEL 1%
#26029 |True|WEIDOS FILM COATED TABLETS 150MG
#26030 |True|WEL HEALTH STRONG SENNA T

#26196 |True|ZITHRIN TABLET 250MG
#26197 |True|Z-NEWDERMA CREAM
#26198 |True|ZOFLAM-100 TABLETS 100MG
#26199 |True|ZOLDOX F.C. TAB 10MG "WEIDAR"
#26200 |True|ZOLEDRONIC ACID ACTAVIS CONCENTRATE FOR SOLUTION FOR INFUSION 4MG/5ML
#26201 |True|ZOLEDRONIC ACID CONCENTRATE FOR SOLUTION FOR INFUSION 4MG/5ML
#26202 |True|ZOLEDRONIC ACID FOR INJ 4MG
#26203 |True|ZOLEDRONIC ACID HOSPIRA CONCENTRATE FOR SOLUTION FOR INFUSION 4MG/5ML
#26204 |True|ZOLEDRONIC ACID INJ 4MG/5ML
#26205 |True|ZOLEDRONIC ACID INJECTION 4MG/5ML
#26206 |True|ZOLEDRONIC ACID-TEVA CONCENTRATE FOR SOLUTION FOR INFUSION 4MG/5ML
#26207 |True|ZOLID TAB 15MG
#26208 |True|ZOLID TAB 30MG
#26209 |True|ZOLID TAB 45MG
#26210 |True|ZOLMAN F.C. TAB 10MG
#26211 |True|ZOLNOX TAB 10MG
#26212 |True|ZOLON FILM COATED TAB 7.5MG
#26213 |True|ZOLPIDEM STADA TABLETS 10MG
#26214 |True|ZOMACTON POWDER AND SOLVENT FOR SOLN FOR INJ 10MG/ML
#26215 |True|ZOMACTON POWDER AND SOLVENT FOR SOLUTION FOR INJ
#26216 |True|ZOMETA SOLUTION FOR INFUSION 4MG/10

In [7]:
# To filter for empty or blank "updated_by":
not_updated_by_drugs = InventoryItem.objects.filter(
    Q(updated_by__isnull=True)|
    Q(updated_by__exact=''))
not_updated_by_drugs.count()

24364

In [8]:
# To filter by "last_updated" is NULL
never_updated_drugs = InventoryItem.objects.filter(
    last_updated__isnull=True
)
never_updated_drugs.count()

24359

In [9]:
# Check if any of the never_updated_drugs have been recorded in prescribed_drugs
for drug in never_updated_drugs:
    if drug.id in prescribed_drugs:
        print(f"Drug #{drug.id} was prescribed but not updated?")
        print(f"---- {drug.product_name} / Qty {drug.stock_qty} / By {drug.updated_by} / {drug.last_updated}")
        
        
# prescription -- id in (1,2,130,37094,40177); corresponding encounter -- id in (5,6,42481,45697)

## 21827 Amlodipine (EMLIP-10) => prescription id 37094 => encounter 42481 => con 37095
## -- No inventory movement log record, perhaps because "filled_in_clinic" was =0?
##

Drug #5151 was prescribed but not updated?
---- DEXTROPHAN TAB 15MG / Qty 0.0 / By None / None
Drug #5398 was prescribed but not updated?
---- DIFENOL GEL 1% / Qty 0.0 / By None / None
Drug #19819 was prescribed but not updated?
---- VIDATAPP TAB / Qty 0.0 / By None / None
Drug #21827 was prescribed but not updated?
---- EMLIP-10 TABLETS 10MG / Qty 0.0 / By  / None


In [11]:
pres_dep_set = prescribed_drugs.union(dep_drugid_set)
print(len(pres_dep_set))

591


In [12]:
# Check which drugs are depleted byt not prescribed
depleted_not_prescribed = dep_drugid_set.difference(prescribed_drugs)

In [13]:
for drug_id in depleted_not_prescribed:
    drug = InventoryItem.objects.get(pk=drug_id)
    print(f"#{drug.id} | {drug.product_name}")

#30082 | QUITIN PEC SYRUP
#4998 | DERZID CREAM 0.1%
#8968 | ISOPTIN SR TAB 240MG SUSTAINED-RELEASE
#30089 | VITAMIN D3 1000IU
#22667 | HYTRIN TAB 1MG
#22924 | KOFIXIR 120 TABLETS 120MG
#22925 | KOFIXIR 180 TABLETS 180MG
#23437 | MELCAM TAB 7.5MG
#30098 | PRALUENT SOLUTION FOR INJECTION IN PREFILLED PEN 75MG/ML
#30099 | BREATHEASY SPRAY
#19988 | ACERYCAL TAB 5MG/10MG
#14108 | PRIMAX EYE DROPS 10MG/ML (BOX)
#2976 | CAMIDEX (NF) CAP
#30112 | ESOMEPRAZOLE ACTAVIS GASTRO-RESISTANT TABLETS 20MG
#30117 | VITAMIN B-6 50MG
#4646 | CURAM POWDER FOR ORAL SUSP 312.5MG/5ML
#30118 | VITAMIN B-6 100MG
#20007 | ACT-HIB VACCINE WITH DILUENT
#19369 | EUROCLOVIR CREAM 5%
#3626 | CHLOZINE TAB 25MG
#24358 | PMS-NYSTATIN SUSP 100000U/ML
#24492 | PRIORIX POWDER FOR INJ VACCINE (LIVE)
#558 | ALLNOL PLUS TAB 300MG
#30127 | TADILECTO TABLETS 20MG
#30128 | VEN-Q TABLETS
#23602 | MONTENE 4 CHEWABLE TAB 4MG
#24372 | PMS-PREGABALIN CAPSULES 75MG
#30135 | TERBUL CREAM 10MG/G
#20793 | BINOZYT POWDER FOR SUSP 200MG/5M

OperationalError: (2006, 'MySQL server has gone away')

In [14]:
count = 1
for drug in used_drugs:
    if drug.updated_by:
        print(f"{count:<5}: id #{drug.id:<7} | {drug.updated_by} | Qty: {drug.stock_qty:<8} | {drug.product_name}")
    #print(f"{count:<5}: id #{drug.id:<7} | Qty: {drug.stock_qty:<8} | {drug.product_name}")
    count += 1

NameError: name 'used_drugs' is not defined

In [9]:
drugs_to_reset = InventoryItem.objects.filter(Q(updated_by__isnull=True) | Q(updated_by=''))

In [14]:
count = 0
count10k = 0
reset_ids = []
for drug in drugs_to_reset:
    if drug.last_updated is not None:
        print(f"ID:{drug.id:<6}| last updated on {drug.last_updated}")
    if drug.discontinue == 0:
        # Set inactive (discontinue=1)
        print(f"ID:{drug.id:<6}| Mstr {drug.is_master_drug_list} Clin {drug.is_clinic_drug_list} | Set inactive: {drug.product_name}")
    if drug.stock_qty == 10000:
        # Reset to 0
        count10k += 1
        reset_ids.append(drug.id)
    elif drug.stock_qty == 0:
        print(f"ID:{drug.id:<6}| Qty already 0: {drug.product_name}")
    else:
        print(f"ID:{drug.id:<6}| Qty[{drug.stock_qty:<7}]: {drug.product_name}")
    count += 1
print(f"Counted: {count}; Counted stock_qty is 10k {count10k}")
print(reset_ids)

ID:1     | last updated on 2020-11-12 10:28:59+00:00
ID:1     | Mstr False Clin False | Set inactive: 0.05% CHLORHEXIDINE & 0.5% CETRIMIDE SOL
ID:1     | Qty[9999.0 ]: 0.05% CHLORHEXIDINE & 0.5% CETRIMIDE SOL
ID:2     | Qty already 0: 0.1% CHLORHEXIDINE & 1% CETRIMIDE SOLN
ID:3     | Qty already 0: 0.12% PRED MILD OPHTH SUSP
ID:4     | Qty already 0: 0.15% POT CHL/0.9% SOD CHL IV INF
ID:5     | Qty already 0: 0.15% POT CHL/5% GLUC IV INFUSION
ID:6     | Qty already 0: 0.15% POT CL/2.5% GLUC/0.45% SOD CL INJ
ID:7     | Qty already 0: 0.15% POTASSIUM CHLORIDE AND 0.9% SODIUM CHLORIDE IV INF
ID:8     | Qty already 0: 0.15% POTASSIUM CHLORIDE AND 5% GLUCOSE IV INF
ID:9     | Qty already 0: 0.15% POTASSIUM CHLORIDE IN 2.5% GLUCOSE AND 0.45% SODIUM CHLORIDE IV INF
ID:10    | Qty already 0: 0.18% SOD CHLORIDE & 4.3% GLUCOSE INJ
ID:11    | Qty already 0: 0.18% SODIUM CHLORIDE 4.3% DEXTROSE INJ
ID:12    | Qty already 0: 0.18% SODIUM CHLORIDE AND 4.3% DEXTROSE INJ
ID:13    | Qty already 0: 0.18%

ID:1559  | Qty already 0: ARFEN PLUS TAB
ID:1560  | Qty already 0: ARFEN TAB 500MG
ID:1561  | Qty already 0: ARGINMAX FOR MEN CAP
ID:1562  | Qty already 0: ARGINMAX FOR WOMEN TAB
ID:1563  | Qty already 0: ARICEPT TAB 10MG
ID:1564  | Qty already 0: ARICEPT TAB 5MG
ID:1565  | Qty already 0: ARIEL TDDS TRANSDERMAL PATCH 1.5MG
ID:1566  | Qty already 0: ARIKAN CAP
ID:1567  | Qty already 0: ARIMIDEX TAB 1MG
ID:1568  | Qty already 0: ARISTOBET-N EYE OINTMENT
ID:1569  | Qty already 0: ARISTOBET-N EYE, EAR & NASAL DROPS
ID:1570  | Qty already 0: ARISTOPHEN EYE DROPS 0.5%
ID:1571  | Qty already 0: ARISTOPHEN EYE OINTMENT 1% W/W
ID:1572  | Qty already 0: ARISTTO SYRUP
ID:1573  | Qty already 0: ARITACIN CREAM
ID:1574  | Qty already 0: ARIXTRA PRE-FILL SYR FOR INJ 2.5MG/0.5ML
ID:1575  | Qty already 0: ARIXTRA SOLUTION FOR INJ 7.5MG/0.6ML
ID:1576  | Qty already 0: ARIZEN CAP
ID:1577  | Qty already 0: ARKET SUSTAINED RELEASE CAP 200MG
ID:1578  | Qty already 0: ARKINAS CAP
ID:1579  | Qty already 0: AR

ID:3126  | Qty already 0: CARVEDILOL TAB 25MG
ID:3127  | Qty already 0: CARVEDILOL TAB 3.125MG
ID:3128  | Qty already 0: CARVEDILOL TAB 6.25MG
ID:3129  | Qty already 0: CARVEDILOL-TEVA TAB 12.5MG
ID:3130  | Qty already 0: CARVEDILOL-TEVA TAB 6.25MG
ID:3131  | Qty already 0: CARYOPIN TAB
ID:3132  | Qty already 0: CARZEPIN TAB 200MG
ID:3133  | Qty already 0: CASBESINTON SYRUP
ID:3134  | Qty already 0: CASHINO GEL 1%
ID:3135  | Qty already 0: CASINO SOLUTION 70% FEMME
ID:3136  | Qty already 0: CASINO SOLUTION 70% MILD FRAGRANCE
ID:3137  | Qty already 0: CASINO SOLUTION 70% REGULAR
ID:3138  | Qty already 0: CASIROL COUGH SYRUP
ID:3139  | Qty already 0: CASIROL COUGH SYRUP (F2)
ID:3140  | Qty already 0: CASODEX TAB 150MG
ID:3141  | Qty already 0: CASODEX TAB 50MG
ID:3142  | Qty already 0: CASOLINE COUGH SYRUP
ID:3143  | Qty already 0: CASSULIE CAP
ID:3146  | Qty already 0: CASULONE CREAM
ID:3147  | Qty already 0: CATAFLAM TAB 25MG
ID:3148  | Qty already 0: CATAFLAM TAB 50MG
ID:3149  | Qty a

ID:4308  | Qty already 0: COLPOTROPHINE GYNAECOLOGICAL CREAM 1%
ID:4309  | Qty already 0: COLSEDYL COUGH SYRUP
ID:4310  | Qty already 0: COLSOR CREAM
ID:4311  | Qty already 0: COLSTATIN CREAM
ID:4312  | Qty already 0: COLTALIN WITH VIT B1 TAB (FOR ADULT)
ID:4313  | Qty already 0: COLTALIN-GP EXTRA TAB
ID:4314  | Qty already 0: COLTALIN-GP TAB
ID:4315  | Qty already 0: COLTALIN-ND TAB
ID:4316  | Qty already 0: COLTRAN CAP
ID:4317  | Qty already 0: COLTRAN JUNIOR TAB
ID:4318  | Qty already 0: COM-CLOX CAP
ID:4319  | Qty already 0: COMBECEN INJ (VET)
ID:4320  | Qty already 0: COMBEVIT TAB
ID:4321  | Qty already 0: COMBIDERM CREAM
ID:4322  | Qty already 0: COMBIGAN EYE DROPS
ID:4323  | Qty already 0: COMBIVENT INHALER
ID:4324  | Qty already 0: COMBIVENT SOL FOR INHALATION 2.5ML VIAL
ID:4325  | Qty already 0: COMBIVIR TAB
ID:4326  | Qty already 0: COMBIZYM TAB
ID:4327  | Qty already 0: COMCLOTAZOL CREAM
ID:4328  | Qty already 0: COMDTMAX CREAM
ID:4329  | Qty already 0: COMEPHEN COUGH SYRUP


ID:5249  | Qty already 0: DHS ZINC SHAMPOO 2%
ID:5250  | Qty already 0: DI-ADRESON F TAB 5MG
ID:5251  | Qty already 0: DIABACT UBT TAB 50MG
ID:5252  | Qty already 0: DIABETIN TAB 5MG
ID:5255  | Qty already 0: DIABO CREAM
ID:5256  | Qty already 0: DIACALM CAP 2MG
ID:5257  | Qty already 0: DIACON TAB 80MG
ID:5260  | Qty already 0: DIACURE TAB 2MG (FLESH)
ID:5261  | Qty already 0: DIAFORMIN TAB 500MG
ID:5262  | Qty already 0: DIAGNOGREEN FOR INJ 25MG/VIAL
ID:5263  | Qty already 0: DIAHCURE CAP 2MG
ID:5264  | Qty already 0: DIAL ANTIBACT HAND SOAP SPRING WATER ENRICHED W/ ALOE 0.2%W/W
ID:5265  | Qty already 0: DIAL ANTIBACTERIAL HAND SOAP MOUNTAIN FRESH ENRICHED W/ALOE 0.19%
ID:5266  | Qty already 0: DIAL ANTIBACTERIAL HAND SOAP TROPICAL ESCAPE W/ALOE 0.19%
ID:5267  | Qty already 0: DIAL ANTIBACTERIAL MOISTURIZING BODY WASH 0.15%W/W
ID:5268  | Qty already 0: DIAL ANTIBACTERIAL WITH VITAMINS HYDRATING BODY WASH 0.15%
ID:5269  | Qty already 0: DIAL DAILY CARE ANTIBACT HAND SOAP ENRICHED W/ V

...

ID:20490 | Qty already 0: APTSEDYL DM COUGH SYRUP
ID:20491 | Qty already 0: APT-SILDENAFIL TABLETS 100MG
ID:20492 | Qty already 0: APT-SILDENAFIL TABLETS 50MG
ID:20493 | Qty already 0: APT-SIMET TABLETS 40MG
ID:20494 | Qty already 0: ARASTAD 20 TABLETS 20MG
ID:20495 | Qty already 0: AREMED 1  TAB 1MG
ID:20496 | Qty already 0: ARI GALLIUM [67GA] CITRATE INJ 100MBQ/ML
ID:20497 | Qty already 0: ARI THALLOUS (201 TL) CHLORIDE INJ 125MBQ/ML
ID:20498 | Qty already 0: ARICEPT EVESS ORODISPERSIBLE TAB 10MG
ID:20499 | Qty already 0: ARICEPT EVESS ORODISPERSIBLE TAB 5MG
ID:20500 | Qty already 0: ARICEPT TABLETS 23MG
ID:20501 | Qty already 0: ARIFAX-20 TAB 20MG
ID:20502 | Qty already 0: ARIPIPRAZOLE SANDOZ TABLETS 10MG
ID:20503 | Qty already 0: ARIPIPRAZOLE SANDOZ TABLETS 15MG
ID:20504 | Qty already 0: ARIPIPRAZOLE SANDOZ TABLETS 5MG
ID:20505 | Qty already 0: ARIPIPRAZOLE TABLETS 10MG
ID:20506 | Qty already 0: ARIPIPRAZOLE TABLETS 15MG
ID:20507 | Qty already 0: ARIPIPRAZOLE TABLETS 5MG
ID:20508 |

ID:21305 | Qty already 0: COENZYME Q10 PLUS CAP
ID:21306 | Qty already 0: COFFREE TABLETS 60MG
ID:21307 | Qty already 0: COFLUKON COUGH SYRUP
ID:21308 | Qty already 0: COLCHICINE TAB 0.5MG
ID:21309 | Qty already 0: COLCHICINE TABLETS 0.5MG
ID:21310 | Qty already 0: COLCHILY TAB 0.6MG
ID:21311 | Qty already 0: COLDOTAPP-N CAP
ID:21312 | Qty already 0: COLDSPAN CAPSULES
ID:21313 | Qty already 0: COLGOUT TAB 0.5MG
ID:21314 | Qty already 0: COLISTIN POWDER FOR SOLUTION FOR INJECTION 150MG
ID:21315 | Qty already 0: COLKOF-DAY CAPSULE
ID:21316 | Qty already 0: COLKU TABLET
ID:21317 | Qty already 0: COLOMYCIN FOR INJ 1 MILLION IU
ID:21318 | Qty already 0: COLSORIN ORAL GEL
ID:21319 | Qty already 0: COMBIVIR TABLETS
ID:21320 | Qty already 0: COMEDIN COUGH SYRUP
ID:21321 | Qty already 0: COMESGEN TAB 500 MCG
ID:21322 | Qty already 0: COMFORTIS CHEWABLE TABLETS 140MG (VET)
ID:21323 | Qty already 0: COMFORTIS CHEWABLE TABLETS 1620MG (VET)
ID:21324 | Qty already 0: COMFORTIS CHEWABLE TABLETS 270MG

ID:22227 | Qty already 0: FOBAN HYDRO OINTMENT
ID:22228 | Qty already 0: FOBAN OINT 2%
ID:22230 | Qty already 0: FOBANCORT OINT
ID:22231 | Qty already 0: FOLAX TAB 10MG
ID:22232 | Qty already 0: FOLCASIN SOLUTION FOR INJECTION 30MG/10ML
ID:22233 | Qty already 0: FOLIC ACID TABLET 1MG (JAMIESON)
ID:22234 | Qty already 0: FOLIC ACID TABLETS 0.4MG (CHANGZHOU PHARM)
ID:22235 | Qty already 0: FOLIC ACID TABLETS 5MG
ID:22236 | Qty already 0: FOLIRON IRON WITH FOLIC ACID CAP
ID:22237 | Qty already 0: FOLLITROPE FOR INJ 150IU
ID:22238 | Qty already 0: FOLLITROPE FOR INJ 75IU
ID:22239 | Qty already 0: FOLLITROPE PREFILLED SYRINGE INJ 150IU/0.3ML
ID:22240 | Qty already 0: FOLLITROPE PREFILLED SYRINGE INJ 225IU/0.45ML
ID:22241 | Qty already 0: FOLLITROPE PREFILLED SYRINGE INJ 300IU/0.6ML
ID:22242 | Qty already 0: FOLLITROPE PREFILLED SYRINGE INJ 75IU/0.15ML
ID:22243 | Qty already 0: FONA CREAM 0.1%
ID:22244 | Qty already 0: FONGITAR LIQUID
ID:22245 | Qty already 0: FORAND ONCE WEEKLY TAB 70MG
ID:

ID:23985 | Qty already 0: ONGLYZA TAB 2.5MG
ID:23986 | Qty already 0: ONGLYZA TAB 5MG
ID:23987 | Qty already 0: ONILENE TABLETS
ID:23988 | Qty already 0: ONZAPIN OD TAB 10MG
ID:23989 | Qty already 0: ONZAPIN OD TAB 5MG
ID:23990 | Qty already 0: ONZAPIN  TAB 10MG
ID:23991 | Qty already 0: ONZAPIN TAB 5MG
ID:23992 | Qty already 0: OPHTAMOLOL EYE DROPS 0.25%
ID:23993 | Qty already 0: OPHTAMOLOL EYE DROPS 0.5%
ID:23994 | Qty already 0: OPIDOL 50 CAP 50MG
ID:23995 | Qty already 0: OPTILIP-40 TAB 40MG
ID:23996 | Qty already 0: OPTISLIM CAPSULES 120MG
ID:23997 | Qty already 0: OPTIVE ADVANCED LUBRICANT EYE DROPS
ID:23998 | Qty already 0: OPTIVE SENSITIVE LUBRICANT EYE DROPS
ID:23999 | Qty already 0: OPTIVEN 10 TAB 10MG
ID:24000 | Qty already 0: OPTIVEN 20 TAB 20MG
ID:24001 | Qty already 0: OPTIZOLINE EYE DROPS,0.05%W/V
ID:24002 | Qty already 0: OPTODEXINE EYE DROPS
ID:24003 | Qty already 0: OPTO-PHARM STERILE WASH
ID:24004 | Qty already 0: ORACAN DENTAL PASTE 1MG/GM
ID:24005 | Qty already 0: 

ID:24846 | Qty already 0: ROSUVASTATIN-TEVA FILM-COATED TABLETS 40MG
ID:24847 | Qty already 0: ROSUVASTATIN-TEVA FILM-COATED TABLETS 5MG
ID:24848 | Qty already 0: ROSUXL 10 TAB 10MG
ID:24849 | Qty already 0: ROSUXL TAB 5MG
ID:24850 | Qty already 0: ROTAMIN HIGH POTENCY B-CO + C TAB
ID:24851 | Qty already 0: ROTARAD INJECTION 1000MCG/ML
ID:24852 | Qty already 0: ROTHRICIN TAB 150MG
ID:24853 | Qty already 0: ROUEE-RIFAMPICIN CAPSULES 300MG
ID:24854 | Qty already 0: ROVASET-10 TABLETS 10MG
ID:24855 | Qty already 0: ROVASET-20 TABLETS 20MG
ID:24856 | Qty already 0: ROVISTA TAB 10MG
ID:24857 | Qty already 0: ROVISTA TAB 20MG
ID:24858 | Qty already 0: ROVISTA TAB 5MG
ID:24859 | Qty already 0: ROVO TAB 1MG
ID:24860 | Qty already 0: ROXITHROMYCIN TAB 150MG
ID:24861 | Qty already 0: ROY GLUCO & CHON PLUS CALCIUM TABLETS
ID:24862 | Qty already 0: ROZEX GEL 0.75%
ID:24863 | Qty already 0: RUBOLIN PLASTER 30MG/PLASTER
ID:24864 | Qty already 0: RUMACUR-120 PLASTER 120MG
ID:24865 | Qty already 0: RU

In [12]:
# Reset stock_qty to 0
for drug in drugs_to_reset:
    if drug.stock_qty == 10000:
        print(drug.registration_no, drug.stock_qty, drug.product_name, drug.clinic_drug_no, drug.date_created, drug.last_updated)
        drug.stock_qty = 0
        drug.is_master_drug_list = False
        drug.is_clinic_drug_list = False
        drug.discontinue = True
        
        drug.save()

HK55323 10000.0 ADRIEN GAGNON CALCIUM MAGNESIUM WITH VITAMIN D AND ZINC TAB None None None
HK51086 10000.0 AMPRA CAP 20MG (ENTERIC-COATED PELLETS) None None None
HK06139 10000.0 ANUSOL OINT None None None
HK36923 10000.0 APO-METOPROLOL 100MG TAB None None None
HK45778 10000.0 BEPROSONE CREAM 0.05% None None None
HK17385 10000.0 BOGEN-50 S C TAB None None None
HK48604 10000.0 BROM-RAMINE CO. SYRUP (NEW FORMULA) None None None
HK20057 10000.0 BROMHEXINE TAB 8MG (VICKMANS) None None None
HK39022 10000.0 BRONCO-DM TAB None None None
HK39174 10000.0 BUCANIN SYRUP 3MG/5ML None None None
HK43233 10000.0 BUPOGESIC 200 TAB 200MG None None None
HK20158 10000.0 CECLOR FOR ORAL SUSP 125MG/5ML None None None
HK54294 10000.0 CLACIN PLUS TAB 500MG None None None
HK50621 10000.0 CLAMOVID 625 TAB 625MG None None None
HK50620 10000.0 CLAMOVID GRANULES 156.25MG/5ML None None None
HK17506 10000.0 COLDEE TAB None None None
HK46922 10000.0 CP-B CO CAPSULE None None None
HK44221 10000.0 DIFENOL GEL 1% None N

In [13]:
check = InventoryItem.objects.get(id=2)
print(check.registration_no, check.stock_qty, check.product_name, check.clinic_drug_no, check.date_created, check.last_updated)

HK42683 0.0 0.1% CHLORHEXIDINE & 1% CETRIMIDE SOLN None None None


In [30]:
check.stock_qty = 10000
print(check.registration_no, check.stock_qty, check.product_name, check.clinic_drug_no, check.date_created, check.last_updated)

HK42683 10000 0.1% CHLORHEXIDINE & 1% CETRIMIDE SOLN None None None


In [31]:
check.save()